In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# !pip install torch torchvision visdom mlflow scikit-image


In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms
from skimage.metrics import peak_signal_noise_ratio, structural_similarity, mean_squared_error
from tqdm import tqdm

In [4]:
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
# Custom Dataset Class
class CustomImageDataset(Dataset):
    def __init__(self, image_dir_raw, image_dir_ref, transform=None):
        self.image_dir_raw = image_dir_raw
        self.image_dir_ref = image_dir_ref
        self.transform = transform
        self.raw_image_filenames = sorted([f for f in os.listdir(image_dir_raw) if os.path.isfile(os.path.join(image_dir_raw, f))])
        self.ref_image_filenames = sorted([f for f in os.listdir(image_dir_ref) if os.path.isfile(os.path.join(image_dir_ref, f))])
        
    def __len__(self):
        return len(self.raw_image_filenames)

    def __getitem__(self, idx):
        raw_img_path = os.path.join(self.image_dir_raw, self.raw_image_filenames[idx])
        ref_img_path = os.path.join(self.image_dir_ref, self.ref_image_filenames[idx])
        
        raw_image = Image.open(raw_img_path).convert("RGB")
        ref_image = Image.open(ref_img_path).convert("RGB")
        
        if self.transform:
            raw_image = self.transform(raw_image)
            ref_image = self.transform(ref_image)
            
        return raw_image, ref_image

In [6]:
# Define the Variational AutoEncoder Model
class VariationalAutoEncoder(nn.Module):
    def __init__(self, features_d):
        super(VariationalAutoEncoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            self._enc_block(3, features_d, 3, 2, 1),
            self._enc_block(features_d, features_d * 2, 3, 2, 1),
            self._enc_block(features_d * 2, features_d * 4, 3, 2, 1),
        )
        # Latent space
        self.conv_mu = nn.Conv2d(features_d * 4, features_d * 4, kernel_size=3, stride=1, padding=1)
        self.conv_logvar = nn.Conv2d(features_d * 4, features_d * 4, kernel_size=3, stride=1, padding=1)
        # Decoder
        self.decoder = nn.Sequential(
            self._dec_block(features_d * 4, features_d * 2, 3, 1, 1),
            self._dec_block(features_d * 2, features_d, 3, 1, 1),
            nn.Upsample(scale_factor=2),
            nn.ConvTranspose2d(features_d, 3, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid(),
        )

    def _enc_block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def _dec_block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu

    def forward(self, imgs):
        x = self.encoder(imgs)
        mu = self.conv_mu(x)
        logvar = self.conv_logvar(x)
        z = self.reparameterize(mu, logvar)
        reconstructed_imgs = self.decoder(z)
        return reconstructed_imgs


In [7]:
# Initialize the model
model = VariationalAutoEncoder(features_d=64).to(device)

In [8]:
# Load Dataset
train_dir = '/kaggle/input/wec-task-2/Train'
test_dir = '/kaggle/input/wec-task-2/Test'

train_raw_dir = os.path.join(train_dir, 'Raw')
train_ref_dir = os.path.join(train_dir, 'Reference')

test_raw_dir = os.path.join(test_dir, 'Raw')
test_ref_dir = os.path.join(test_dir, 'Reference')

In [9]:
import torchvision.transforms as transforms

# Define the transform with resizing to (480, 640)
transform = transforms.Compose([
    transforms.Resize((480, 640)),  # Resize to fixed size
    transforms.ToTensor()           # Convert image to tensor
])

In [10]:
train_dataset = CustomImageDataset(image_dir_raw=train_raw_dir, image_dir_ref=train_ref_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

test_dataset = CustomImageDataset(image_dir_raw=test_raw_dir, image_dir_ref=test_ref_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [11]:
# Training Function
def train_model(model, train_loader, test_loader, epochs=10, lr=0.001):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for noisy, clean in tqdm(train_loader):
            noisy, clean = noisy.to(device), clean.to(device)
            optimizer.zero_grad()
            reconstructed = model(noisy)
            loss = criterion(reconstructed, clean)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Evaluate on the test set
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for noisy, clean in test_loader:
                noisy, clean = noisy.to(device), clean.to(device)
                reconstructed = model(noisy)
                loss = criterion(reconstructed, clean)
                test_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss/len(train_loader)}, Test Loss: {test_loss/len(test_loader)}")


In [12]:
# Start training
train_model(model, train_loader, test_loader, epochs=100, lr=0.001)

100%|██████████| 88/88 [01:16<00:00,  1.15it/s]


Epoch [1/100], Train Loss: 0.03956297490830449, Test Loss: 0.03996192322423061


100%|██████████| 88/88 [01:00<00:00,  1.45it/s]


Epoch [2/100], Train Loss: 0.027369645339521496, Test Loss: 0.02728709625080228


100%|██████████| 88/88 [01:01<00:00,  1.42it/s]


...

Epoch [98/100], Train Loss: 0.014466086619491265, Test Loss: 0.01949530156950156


100%|██████████| 88/88 [00:58<00:00,  1.51it/s]


Epoch [99/100], Train Loss: 0.014296139897355302, Test Loss: 0.020464008674025536


100%|██████████| 88/88 [00:58<00:00,  1.51it/s]


Epoch [100/100], Train Loss: 0.01480963271619244, Test Loss: 0.01877915777731687


In [13]:
import os
from PIL import Image

In [14]:
def evaluate_model(model, test_loader, output_dir="output_images"):
    model.eval()

    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Initialize lists to store results
    psnr_list = []
    ssim_list = []
    mse_list = []

    with torch.no_grad():
        for batch_idx, (noisy, clean) in enumerate(test_loader):
            noisy, clean = noisy.to(device), clean.to(device)
            reconstructed = model(noisy)
            
            # Iterate through the batch to process each image individually
            for i in range(reconstructed.size(0)):  # Loop over batch size
                output_np = reconstructed[i].cpu().numpy().transpose(1, 2, 0)
                clean_np = clean[i].cpu().numpy().transpose(1, 2, 0)

                # Save reconstructed image
                reconstructed_img = (output_np * 255).astype(np.uint8)  # Convert to uint8 for saving
                img = Image.fromarray(reconstructed_img)
                img_save_path = os.path.join(output_dir, f"enhanced_{batch_idx}_{i}.png")
                img.save(img_save_path)
                
                # Calculate PSNR
                psnr = peak_signal_noise_ratio(clean_np, output_np, data_range=1.0)
                
                # Calculate SSIM with explicit data_range and win_size
                ssim = structural_similarity(clean_np, output_np, multichannel=True, win_size=3, data_range=1.0)
                
                # Calculate MSE
                mse = mean_squared_error(clean_np, output_np)

                # Append the results
                psnr_list.append(psnr)
                ssim_list.append(ssim)
                mse_list.append(mse)

        print(f'Average PSNR: {sum(psnr_list)/len(psnr_list):.4f}')
        print(f'Average SSIM: {sum(ssim_list)/len(ssim_list):.4f}')
        print(f'Average MSE: {sum(mse_list)/len(mse_list):.4f}')


In [15]:
# Evaluate the model
evaluate_model(model, test_loader)

Average PSNR: 18.7001
Average SSIM: 0.6258
Average MSE: 0.0187
